In [1]:
!pip install dash==1.19.0  

!pip install jupyter_dash 

!pip install --upgrade plotly

  Created wheel for dash: filename=dash-1.19.0-py3-none-any.whl size=84021 sha256=9546fcdbbd3325b9e24004fa7cb4c9020acd65e054159f671b24529b4ed15063
  Stored in directory: c:\users\supat\appdata\local\pip\cache\wheels\d3\a4\51\d5de70e3fbed7165091a1972b5d60522f078d8f3b0938c6bc6
  Created wheel for dash-core-components: filename=dash_core_components-1.15.0-py3-none-any.whl size=3527013 sha256=46b8eec5a45c1d5a6db5974bd5235692c114f6769843a6b6dd28750a3e652bdb
  Stored in directory: c:\users\supat\appdata\local\pip\cache\wheels\17\6f\34\e5d7fe1c7189531b3c6c90266df517f73210b9ccf6aabd9c53
  Created wheel for dash-html-components: filename=dash_html_components-1.1.2-py3-none-any.whl size=427844 sha256=fadada7279d11f69744b8e56accbb10575897f08158b56a23bc31b80e4f781ec
  Stored in directory: c:\users\supat\appdata\local\pip\cache\wheels\89\69\97\d277cdcfc9018c0c50a181a3572895c323f324ce30616e988e
  Created wheel for dash-renderer: filename=dash_renderer-1.9.0-py3-none-any.whl size=1014868 sha256=e8d8b

In [34]:
# Import required libraries
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
from jupyter_dash import JupyterDash
import plotly.graph_objects as go
from dash.dependencies import Input, Output
import plotly.express as px

In [54]:
# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# Create a dash application
app = JupyterDash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                # dcc.Dropdown(id='site-dropdown',...)
                                dcc.Dropdown(id='site-dropdown',
                                            options=[
                                                {'label': 'All Sites', 'value': 'ALL'},
                                                {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                                                {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
                                                {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                                                {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'}
                                            ],
                                            value='ALL',
                                            placeholder="place holder here",
                                            searchable=True
                                            ),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                #dcc.RangeSlider(id='payload-slider',...)
                                dcc.RangeSlider(id='payload-slider',
                                                min=0, max=10000, step=1000,
                                                marks={0: '0',
                                                       2500: '2500',
                                                       5000: '5000',
                                                       7500: '7500',
                                                       10000: '10000'},
                                                value=[min_payload, max_payload]),
                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))
def get_pie_chart(entered_site):
    if entered_site == 'ALL':
        fig = px.pie(spacex_df, values='class', 
        names='Launch Site', 
        title='Total Success Launches By Sites')
        return fig
    else:
        # return the outcomes piechart for a selected site
        filtered_df = spacex_df.loc[(spacex_df['Launch Site']==entered_site), ['Launch Site', 'class']].groupby(by='class', as_index=False).count()
        fig = px.pie(filtered_df, values='Launch Site', 
        names='class', 
        title='Total Success Launches for Site ' + str(entered_site))
        return fig
# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
              [Input(component_id='site-dropdown', component_property='value'), 
               Input(component_id="payload-slider", component_property="value")])
def get_scatter_chart(entered_site, payload):
    filtered_df = spacex_df[(spacex_df['Payload Mass (kg)'] >= payload[0]) & 
                            (spacex_df['Payload Mass (kg)']<= payload[1])]
    if entered_site == 'ALL':
        fig = px.scatter(filtered_df, x='Payload Mass (kg)', y='class', color='Booster Version Category',
        title='Correlation between Payload and Success for all Sites')
        return fig
    else:
        # return the outcomes piechart for a selected site
        fig = px.scatter(filtered_df[filtered_df['Launch Site']==entered_site], 
                         x='Payload Mass (kg)', y='class', color='Booster Version Category',
                        title='Correlation between Payload and Success for Site ' + str(entered_site)) 
        return fig
# Run the app
if __name__ == '__main__':
    app.run_server(mode='jupyterlab', port = 8090 ,dev_tools_ui=True, #debug=True,
              dev_tools_hot_reload =True, threaded=True)
    
# run in browser use the url http://localhost:8090